In [18]:
# !pip install "cloud-sql-python-connector[pg8000]"
# !pip install psycopg2

In [19]:
import psycopg2
from psycopg2 import extras
import pandas as pd
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_colwidth', None)
import altair as alt

In [20]:
# Replace these with your actual details
user="xxxxx"
password="xxxxx"
database = "cfpb"
host = "xx.xx.xx.xx"

# Connect to your postgres server
conn = psycopg2.connect(
    dbname=database,
    user=user,
    password=password,
    host=host
)
# Create a cursor object
cur = conn.cursor()

In [21]:
# # Method 1
# # Fetch data in chunks
# chunk_size = 10000  # you can adjust this value depending on your system's memory and requirements
# chunks = []
# for chunk in pd.read_sql_query(query, conn, chunksize=chunk_size):
#     # process data in chunks as needed, or save each chunk for later processing
#     chunks.append(chunk)

# # Combine chunks into one DataFrame
# df = pd.concat(chunks)


# # Method 2
# # Execute the query
# cur.execute(query)
# # Fetch all rows
# rows = cur.fetchall()
# # Get the column names
# colnames = [desc[0] for desc in cur.description]
# # Convert to DataFrame
# df = pd.DataFrame(rows, columns=colnames)

# Method 3
# df = pd.read_sql_query(query, conn)

In [22]:
# get the basic idea in the table cause we changed the namings
query = f"""
SELECT *
FROM cfpb
ORDER BY random()
LIMIT 100;
"""
df = pd.read_sql_query(query, conn)
print(df.columns)
df

C:\Users\JYM\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Index(['id', 'Date_received', 'Product', 'Sub_product', 'Issue', 'Sub_issue',
       'Consumer_complaint_narrative', 'Company_public_response', 'Company',
       'State', 'ZIP_code', 'Tags', 'Consumer_consent_provided',
       'Submitted_via', 'Date_sent_to_company', 'Company_response_to_consumer',
       'Timely_response', 'Consumer_disputed', 'Complaint_ID'],
      dtype='object')


,id,Date_received,Product,Sub_product,Issue,Sub_issue,Consumer_complaint_narrative,Company_public_response,Company,State,ZIP_code,Tags,Consumer_consent_provided,Submitted_via,Date_sent_to_company,Company_response_to_consumer,Timely_response,Consumer_disputed,Complaint_ID
0,2599168,2012-09-25,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,Ocwen Financial Corporation,MI,48382,NaN,NaN,Fax,2012-09-25,Closed with explanation,Yes,No,158797
1,2106932,2022-07-01,Debt collection,Medical debt,Written notification about debt,Didn't receive enough information to verify debt,NaN,NaN,Central Portfolio Control Inc.,FL,32824.0,NaN,Consent not provided,Web,2022-07-01,Closed with explanation,Yes,NaN,5727823
2,3354682,2022-10-31,"Credit reporting, credit repair services, or other personal consumer reports",Credit reporting,Incorrect information on your report,Personal information incorrect,"Delete the following information off my credit report. \n\nXXXX XXXX XXXX, XXXX XXXX XXXX, XXXX XXXX XXXX XXXX XXXX XXXX XXXX, XXXX PA XXXX XXXX XXXX XXXX XXXX, XXXX, PA XXXX XXXXXXXX XXXX XXXX XXXX, XXXX, PA XXXX XXXX",Company has responded to the consumer and the CFPB and chooses not to provide a public response,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",PA,19124.0,NaN,Consent provided,Web,2022-10-31,Closed with non-monetary relief,Yes,NaN,6142634
3,1791684,2014-03-05,Consumer Loan,Vehicle loan,Managing the loan or lease,NaN,NaN,NaN,SYNCHRONY FINANCIAL,FL,33594.0,NaN,NaN,Web,2014-03-05,Closed with non-monetary relief,Yes,No,745273
4,1906932,2021-04-21,"Credit reporting, credit repair services, or other personal consumer reports",Credit reporting,Incorrect information on your report,Account status incorrect,I have disputed an account with XXXX on Experian for the past six months to no avail. I have furnished to Experian a letter from said company stating that the account in question is closed and they refuse to correct my credit report.,Company has responded to the consumer and the CFPB and chooses not to provide a public response,Experian Information Solutions Inc.,MD,21045.0,Servicemember,Consent provided,Web,2021-04-21,Closed with non-monetary relief,Yes,NaN,4312283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,67641,2020-06-16,"Credit reporting, credit repair services, or other personal consumer reports",Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,Company has responded to the consumer and the CFPB and chooses not to provide a public response,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",FL,33178.0,NaN,Other,Web,2020-06-16,Closed with explanation,Yes,NaN,3701263
96,3833166,2023-05-04,Debt collection,I do not know,Attempts to collect debt not owed,Debt is not yours,NaN,NaN,"Affirm Holdings, Inc",TX,77545,NaN,Consent not provided,Web,2023-05-04,Closed with explanation,Yes,NaN,6925555
97,1428845,2021-10-22,Checking or savings account,Checking account,Managing an account,Deposits and withdrawals,"On XX/XX/XXXX an unauthorized transaction for {$30000.00} was made from my business account at Chase Bank to a XXXX account that was opened under my name by an unauthorized individual. \nAfter I discovered the fraudulent ACH debit, I reported it to Chase immediately and filed a police report. with the XXXX XXXX Police Department XXXX XXXX XXXX. \nI closed the Chase account from where the funds were stolen, opened a new account, changed my login and password information and implemented security features. A day or two after, there was a second attempt, this time for {$8000.00} ( from the new account ), thanks to the security features that I had put in place, I was able to reject the transaction. \nI submitted a sworn affidavit to Chase Claims Department. A temporary credit was issued to my account ; however, it was reversed on XX/XX/XXXX. Chase claims that their Claims Dept. has disputed the transaction twice and that XXXX has reaffirmed the transaction. I sp

In [23]:
# get unique 'Product' from cfpb and rank them by counts
# for case sentitive column names use doubl quoats for postgre sql 
def get_product():
    query = """
    SELECT 
        "Product", 
        COUNT("Product") AS ProductCount
    FROM 
        cfpb
    GROUP BY 
        "Product"
    ORDER BY 
        ProductCount DESC;
    """
    df = pd.read_sql_query(query, conn)
    return df.Product.to_list()
get_product()

C:\Users\JYM\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


['Credit reporting, credit repair services, or other personal consumer reports',
 'Debt collection',
 'Mortgage',
 'Credit card or prepaid card',
 'Checking or savings account',
 'Credit reporting',
 'Credit card',
 'Bank account or service',
 'Student loan',
 'Money transfer, virtual currency, or money service',
 'Vehicle loan or lease',
 'Consumer Loan',
 'Payday loan, title loan, or personal loan',
 'Payday loan',
 'Money transfers',
 'Prepaid card',
 'Other financial service',
 'Virtual currency']

In [29]:
# get companies had complaints with selected products
def get_companies(product_lst):
    # Prepare the SQL query
    query = """
    SELECT 
        "Company", 
        COUNT("Company") AS CompanyCount
    FROM 
        cfpb
    WHERE 
        "Product" IN ({})
    GROUP BY 
        "Company"
    ORDER BY 
        CompanyCount DESC;
    """

    # Format the query string with the product list
    # Convert each product in product_lst into a string enclosed by single quotes
    formatted_product_lst = ", ".join("'{}'".format(p) for p in product_lst)
    query = query.format(formatted_product_lst)

    # Execute the query and get the data
    df = pd.read_sql_query(query, conn)
    
    # Return the company list
    return df.Company.to_list()

get_companies(['Credit reporting, credit repair services, or other personal consumer reports'])

C:\Users\JYM\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


['EQUIFAX, INC.',
 'TRANSUNION INTERMEDIATE HOLDINGS, INC.',
 'Experian Information Solutions Inc.',
 'CAPITAL ONE FINANCIAL CORPORATION',
 'SYNCHRONY FINANCIAL',
 'LEXISNEXIS',
 'CITIBANK, N.A.',
 'JPMORGAN CHASE & CO.',
 'WELLS FARGO & COMPANY',
 'BANK OF AMERICA, NATIONAL ASSOCIATION',
 'Bread Financial Holdings, Inc.',
 'DISCOVER BANK',
 'PORTFOLIO RECOVERY ASSOCIATES INC',
 'ALLY FINANCIAL INC.',
 'AMERICAN EXPRESS COMPANY',
 'Santander Consumer USA Holdings Inc.',
 'TD BANK US HOLDING COMPANY',
 'Resurgent Capital Services L.P.',
 'Fidelity National Information Services, Inc. (FNIS)',
 'CORELOGIC INC',
 'Navient Solutions, LLC.',
 'NAVY FEDERAL CREDIT UNION',
 'ENCORE CAPITAL GROUP INC.',
 'Nelnet, Inc.',
 'BARCLAYS BANK DELAWARE',
 'ID Analytics, Inc.',
 'CBC Companies, Inc.',
 'U.S. BANCORP',
 'AES/PHEAA',
 'General Motors Financial Company, Inc.',
 'CREDIT ACCEPTANCE CORPORATION',
 'I.C. System, Inc.',
 'ERC',
 'Early Warning Services, LLC',
 'TRUIST FINANCIAL CORPORATION',
 '

In [30]:
# get data
import pandas as pd

def get_filtered_data(product_lst, company_lst, days_of_hist):
    # Prepare the SQL query
    query = """
    SELECT 
        "Date_received",
        "Product", 
        "Sub_product", 
        "Issue", 
        "Sub_issue",
        "Consumer_complaint_narrative", 
        "Company_public_response", 
        "Company",
        "State", 
        "ZIP_code", 
        "Tags", 
        "Consumer_consent_provided",
        "Company_response_to_consumer",
        "Timely_response", "Consumer_disputed", "Complaint_ID"
        
    FROM 
        cfpb
    WHERE 
        "Product" IN ({})
    AND 
        "Company" IN ({})
    AND
        "Date_received" >= CURRENT_DATE - INTERVAL '{} days'
    AND 
        "Consumer_disputed" IN ('Yes', 'No')
    LIMIT 100;
    """

    # Format the query string with the product and company list
    formatted_product_lst = ", ".join("'{}'".format(p) for p in product_lst)
    formatted_company_lst = ", ".join("'{}'".format(c) for c in company_lst)
    query = query.format(formatted_product_lst, formatted_company_lst, days_of_hist)

    # Execute the query and get the data
    df = pd.read_sql_query(query, conn)
    
    return df
get_filtered_data(['Credit reporting, credit repair services, or other personal consumer reports'], ['EQUIFAX, INC.'], 1200)

C:\Users\JYM\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Date_received,Product,Sub_product,Issue,Sub_issue,Consumer_complaint_narrative,Company_public_response,Company,State,ZIP_code,Tags,Consumer_consent_provided,Company_response_to_consumer,Timely_response,Consumer_disputed,Complaint_ID


In [32]:
def check():
    # Prepare the SQL query
    query = """
    SELECT 
        DISTINCT ("Consumer_disputed","Company_response_to_consumer"),
        COUNT("Consumer_disputed") AS Consumer_disputedCount
    FROM 
        cfpb
    GROUP BY 
        "Consumer_disputed"
    ORDER BY 
        Consumer_disputedCount DESC;
    """

    # Format the query string with the product and company list
#     formatted_product_lst = ", ".join("'{}'".format(p) for p in product_lst)
#     formatted_company_lst = ", ".join("'{}'".format(c) for c in company_lst)
#     query = query.format(formatted_product_lst, formatted_company_lst, days_of_hist)

    # Execute the query and get the data
    df = pd.read_sql_query(query, conn)
    
    return df
check()

C:\Users\JYM\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Consumer_disputed,consumer_disputedcount
0,NaN,3132954
1,No,619980
2,Yes,148378


In [4]:
# Close the cursor and connection
cur.close()
conn.close()